## 本notebook提供ChatYuan-large模型使用示例。
包括安装包；加载模型；问答、写作与功能型助手功能，多轮对话的演示。
加微信入讨论群<a href='https://github.com/clue-ai/ChatYuan#%E6%8A%80%E6%9C%AF%E4%BA%A4%E6%B5%81%E5%92%8C%E9%97%AE%E9%A2%98%E5%8F%8D%E9%A6%88'>讨论群</a>

##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys
def find_repo_root():
    try:
        script_path = os.path.abspath('.')
    except NameError:
        script_path = os.path.abspath(__file__)
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    result = None
    while True:
        script_path = os.path.dirname(script_path)
        if os.path.isdir(os.path.join(script_path, '.git')):
            result = script_path
        if script_path == '/':
            return result
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
%cd $repo_root_dir
root_dir = os.path.dirname(repo_root_dir)
# 装载nbfunc中的hub涉及的所有ipynb函数
%run ./nbfunc/hub.ipynb

In [ ]:
clone_and_install("https://github.com/huggingface/transformers.git", "/output/")

In [ ]:
pip_install(
    "sentencepiece",
)

apt_install(
    "iputils-ping",
)

In [ ]:
# 搜索input中的zst文件，解压到指定的目录（中间会产生tar中间产物）
import os

cache_dir = "/output/huggingface/hub"

zst_filename = 'models--ClueAI--ChatYuan-large-v1.tar.zst'
zst_filename_no_ext = remove_extension(zst_filename)
print('zst_filename_no_ext: ',zst_filename_no_ext)

zstd_path = find_file_in_input(zst_filename)
target_path = os.path.join(cache_dir,zst_filename_no_ext)

zstd_unzip(
    zstd_path = zstd_path,
    target_path = target_path
)

In [ ]:
# 加载模型
from transformers import T5Tokenizer,T5ForConditionalGeneration
# 指定模型的安装目录，如果没有对应的目录，则自动下载
tokenizer = T5Tokenizer.from_pretrained("ClueAI/ChatYuan-large-v1", cache_dir=cache_dir)
model = T5ForConditionalGeneration.from_pretrained("ClueAI/ChatYuan-large-v1", cache_dir=cache_dir)

In [ ]:
# 使用
import torch
from transformers import AutoTokenizer
# 修改colab笔记本设置为gpu，推理更快
device = torch.device('cuda')
model.to(device)
def preprocess(text):
  text = text.replace("\n", "\\n").replace("\t", "\\t")
  return text

def postprocess(text):
  return text.replace("\\n", "\n").replace("\\t", "\t")

def answer(text, sample=True, top_p=1, temperature=0.7):
  '''sample：是否抽样。生成任务，可以设置为True;
  top_p：0-1之间，生成的内容越多样'''
  text = preprocess(text)
  encoding = tokenizer(text=[text], truncation=True, padding=True, max_length=768, return_tensors="pt").to(device) 
  if not sample:
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_new_tokens=512, num_beams=1, length_penalty=0.6)
  else:
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_new_tokens=512, do_sample=True, top_p=top_p, temperature=temperature, no_repeat_ngram_size=3)
  out_text = tokenizer.batch_decode(out["sequences"], skip_special_tokens=True)
  return postprocess(out_text[0])
print("end...")

In [ ]:
input_text= "写一个python程序"
print(f"示例".center(50, "="))
input_text = "用户：" + input_text + "\n小元："
output_text = answer(input_text, sample=True)
print(f"{input_text}{output_text}")

In [ ]:
input_text= "你的名字叫小元？可以解释一下它的由来吗？"
print(f"示例".center(50, "="))
input_text = "用户：" + input_text + "\n小元："
output_text = answer(input_text, sample=True)
print(f"{input_text}{output_text}")

# 问答、写作与功能型助手

In [ ]:
input_text0 = "用英文帮我写一个请假条，我因为新冠不舒服，需要请假3天，请领导批准"
input_text1 = "你能干什么"
input_text2 = "用英文写一封道歉的邮件，表达因为物流延误，不能如期到达，我们可以赔偿贵公司所有损失"
input_text3 = "写一个文章，题目是未来城市"
input_text4 = "写一个诗歌，关于冬天"
input_text5 = "从南京到上海的路线"
input_text6 = "学前教育专业岗位实习中，在学生方面会存在问题，请提出改进措施。800字"
input_text7 = "根据标题生成文章：标题：屈臣氏里的化妆品到底怎么样？正文：化妆品，要讲究科学运用，合理搭配。屈臣氏起码是正品连锁店。请继续后面的文字。"
input_text8 = "帮我对比几款GPU，列出详细参数对比，并且给出最终结论"
input_list = [input_text0, input_text1, input_text2, input_text3, input_text4, input_text5, input_text6, input_text7, input_text8]
for i, input_text in enumerate(input_list):
  input_text = "用户：" + input_text + "\n小元："
  print(f"示例{i}".center(50, "="))
  output_text = answer(input_text)
  print(f"{input_text}{output_text}")

# 多轮对话

In [ ]:
input_text = ["你好","新冠什么症状？","可以吃什么药？"]
answer_text = ["您好!我是元语AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息", "新冠是指新型冠状病毒，其症状包括发热、干咳、乏力、嗅味觉减退、呼吸困难等。", "根据您提供的病史，目前没有明确的抗新冠病毒的药物，建议您在家进行自我隔离，避免与他人接触，多喝开水，清淡易消化饮食，避免熬夜和过度劳累，适当进行户外活动。"]

In [ ]:
context = "\n".join([f"用户：{input_text[i]}\n小元：{answer_text[i]}" for i in range(len(input_text))])

In [ ]:
print(context)

In [ ]:
input_text = "有什么后遗症么？"
print(f"示例".center(50, "="))
input_text = context + "\n用户：" + input_text + "\n小元："
output_text = answer(input_text, top_p=0.9)
print(f"{input_text}{output_text}")